In [74]:
# imports and link kismet xml files
import re, pandas as pd, xml.etree.ElementTree as ET
import folium
from folium import FeatureGroup, LayerControl, Map, Marker

tree = ET.parse('Kismet-20191011-18-31-55-1.netxml')
gps_tree = ET.parse('Kismet-20191011-18-31-55-1.gpsxml')
root, gps_root = tree.getroot(), gps_tree.getroot()

In [75]:
########## Find MAC Address Vendor Using OUI Lookup File ##########

file = open('mac-vendors.txt', 'r', encoding="utf8")
mac_dict = {line[:8]: line[9:].strip() for line in file}
def getDeviceType(bssid): return mac_dict.get(bssid[:8], "Unknown")

In [76]:
# netxml parser that creates array data for Wi-Fi devices

def appendArray(net, child):
    if (child.tag == net) :
        bssid, essid = "", ""
        gps, time, encryption = [], [], []
        scan = 'BSSID' if net == "wireless-network" else 'client-mac'
        for bssid_tag in child.findall(scan): bssid = bssid_tag.text
        for gps_tag in child.findall("gps-info"):
            gps = [[float(gps_tag[0].text), float(gps_tag[1].text)],[float(gps_tag[4].text), float(gps_tag[5].text)]]
        time = [child.attrib['first-time'], child.attrib['last-time']]
        for ssid_tag in child.findall('SSID'):
            for essid_tag in ssid_tag.findall('essid'): essid= essid_tag.text
            encryption = [info_tag.text for info_tag in ssid_tag if info_tag.tag == "encryption"]
        StationList.append([bssid,gps,time,getDeviceType(bssid),encryption,essid])  # add individual device entry

In [77]:
########## Create Array of Discovered WiFi Devices ##########
StationList = [] # MAC, GPS, First Seen, Last Seen, Type, Encryption, SSID
for child in root:
    appendArray("wireless-network",child) # network
    for subchild in child:
        appendArray("wireless-client", subchild) # client under network tag

### creep detecting starts here

In [80]:
from geopy.distance import geodesic

# creep list with threshold of .20 miles
threshold = .2
creeps = [StationList[index] for index in range(0,len(StationList)) if geodesic((StationList[index][1][0][0],StationList[index][1][0][1]),(StationList[index][1][1][0],StationList[index][1][1][1])).miles > threshold]

# remove duplicates
unique, duplicate = [], []
for creep, i in zip(creeps, range(len(creeps))) :
    if creep[0] not in unique: unique.append(creep[0])
    else: duplicate.append(i)
for index in sorted(duplicate, reverse=True): del creeps[index]

# check source MAC
for child in creeps : child[1] = [[float(gpss.attrib["lat"]),float(gpss.attrib["lon"])] for gpss in gps_root if (('source' in gpss.attrib) and gpss.attrib['source']==child[0])]

pd.DataFrame(creeps, columns=["BSSID", "GPS","Dates","Manufacturer","Encryption","ESSID"])

,BSSID,GPS,Dates,Manufacturer,Encryption,ESSID
0,00:0D:97:00:5B:33,"[[34.196182, -118.340149], [34.196323, -118.34...","[Fri Oct 11 18:35:27 2019, Fri Oct 11 18:45:34...",ABB Inc./Tropos,[None],BWP Free WiFi
1,00:0D:97:00:C8:C6,"[[34.195801, -118.340218], [34.195702, -118.34...","[Fri Oct 11 18:33:44 2019, Fri Oct 11 18:37:09...",ABB Inc./Tropos,[None],BWP Free WiFi
2,00:0D:97:09:B1:A4,"[[34.19574, -118.34005], [34.190941, -118.3417...","[Fri Oct 11 18:36:36 2019, Fri Oct 11 18:45:43...",ABB Inc./Tropos,[None],None
3,8C:45:00:0C:F2:95,"[[34.196125, -118.34024], [34.196316, -118.340...","[Fri Oct 11 18:32:48 2019, Fri Oct 11 18:45:39...",Unknown,[],
4,B0:72:BF:EE:15:42,"[[34.196342, -118.340179], [34.196342, -118.34...","[Fri Oct 11 18:32:28 2019, Fri Oct 11 18:43:30...","Murata Manufacturing Co., Ltd.",[],
5,B6:E6:2D:53:20:76,"[[34.196041, -118.340248], [34.195942, -118.34...","[Fri Oct 11 18:33:13 2019, Fri Oct 11 18:45:24...",Unknown,"[WPA+TKIP, WPA+PSK, WPA+AES-CCM]",Creeper


In [ ]:
########## Route Approximation ##########

from geopy.distance import geodesic

def sortNearest(coordinates):
    def findLargestGap(coords):
        largestGap = 0
        for i in range(len(coords) - 1):
            distance = geodesic(coords[i], coords[i + 1]).miles
            if distance > largestGap:
                largestGap = distance
                start = i
        return start
    start = 0
    num_coordinates = len(coordinates)

    ordered_coordinates = [coordinates[start]]
    coordinates.pop(start)
    while coordinates:
        nearest = min(coordinates, key=lambda x: geodesic(ordered_coordinates[-1], x).miles)
        ordered_coordinates.append(nearest)
        coordinates.remove(nearest)

    # Find the largest gap
    start = findLargestGap(ordered_coordinates)
    ordered_coordinates = ordered_coordinates[start:] + ordered_coordinates[:start]

    return ordered_coordinates


gps_coordinates = list(set(map(tuple,[[float(child.attrib['lat']),float(child.attrib['lon'])] for child in gps_root if child.tag == "gps-point"])))
sorted_path = sortNearest(gps_coordinates)

print(sorted_path)

In [82]:
# map reference frame, plot path
LDN_COORDINATES = (34.196041, -118.340248)
myMap = folium.Map(location=LDN_COORDINATES, zoom_start=15)
folium.PolyLine(sorted_path,line_opacity = 0.5, weight = 4).add_to(myMap)
colors = [ 'red', 'blue', 'green', 'orange', 'purple', 'beige', 'gray', 'pink', 'black', 'lightgreen', 'darkblue', 'lightblue', 'lightred', 'darkpurple', 'darkred', 'cadetblue', 'lightgray', 'darkgreen' ]

# hacky solution to plot creeps
for x in range(0, len(creeps)):
    globals()['creep%s' % x] = FeatureGroup(name = 'Creep '+ str(x))
    for creep_loc in creeps[x][1]: Marker(location=creep_loc, popup=creeps[x][0],  icon=folium.Icon(color=colors[x % len(colors)], icon='user-secret', prefix='fa')).add_to(globals()['creep%s' % x])
    globals()['creep%s' % x].add_to(myMap)

# add toggle for creeps
LayerControl().add_to(myMap)
myMap